In [ ]:
from statsmodels import regression
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import statsmodels.stats.outliers_influence as smo
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
import scipy.stats as stats

**Ordinary Least Square Regression**<br>

Assume residuals are independently, normally distributed with zero mean and unit variance.<br>

In GLS, the following error term is minimized:

$ (\mathbf {y} -\mathbf {X} \mathbf {b} )^{\mathtt {T}}\,\mathbf {\Omega } ^{-1}(\mathbf {y} -\mathbf {X} \mathbf {b}) $ , where $\Omega $ is the covariance matrix of residuals. 

In OLS, $\Omega $ = $I$, so the error term to be minimized is: <br>

$ (\mathbf {y} -\mathbf {X} \mathbf {b} )^{\mathtt {T}}\,\mathbf (\mathbf {y} -\mathbf {X} \mathbf {b}) $

In [ ]:
def linreg(X,Y):
    # Running the linear regression
    X = sm.add_constant(X)
    model = regression.linear_model.OLS(Y, X).fit() #model the cov matrix of residual by OLS.fit(cov_type='HC1')
    return model #or return model.get_robustcov_results(cov_type='HC1').summary()

**Regression plot and Residuls plot**

In [ ]:
def reg_resid_plot(model,X,Y,x_label,y_label):
    f,ax = plt.subplots(1,2)
    plt.figure(figsize=(50,50))
    
    a = model.params[0]
    b = model.params[1]

    # Return summary of the regression and plot results
    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = X2 * b + a
    ax[0].scatter(X, Y, alpha=0.3) # Plot the raw data
    ax[0].plot(X2, Y_hat, 'r', alpha=0.9);  # Add the regression line, colored in red
    plt.setp(ax[0], xlabel=x_label)
    plt.setp(ax[0], ylabel=y_label)
    ax[0].set_title('Regression')
    
    ax[1].scatter(model.predict(), model.resid, alpha=0.3);
    ax[1].axhline(0, color='red')
    plt.setp(ax[1], xlabel='Predicted_'+ y_label)
    plt.setp(ax[1], ylabel=y_label)
    ax[1].set_title('Residual plot')
    
    f.set_figheight(4)
    f.set_figwidth(16)

In [ ]:
def reg_plot(model,X,Y,x_label,y_label):
    a = model.params[0]
    b = model.params[1]

    # Return summary of the regression and plot results
    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = X2 * b + a
    plt.scatter(X, Y, alpha=0.3) # Plot the raw data
    plt.plot(X2, Y_hat, 'r', alpha=0.9);  # Add the regression line, colored in red
    plt.xlabel(x_label)
    plt.ylabel(y_label)

In [ ]:
def resid_plot(model,y_label):
    plt.scatter(model.predict(), model.resid,alpha=0.3);
    plt.axhline(0, color='red')
    plt.xlabel('Predicted_'+ y_label);
    plt.ylabel('Residuals');

**Weighted Least Square Regression**<br>

Used when residuals are not correlated, but variance of residuals is not constant. <br>

A weight is applied to each error term, the function to be minimized as follows:<br>

$ \sum _{i=1}^{m}w_{i}\left|y_{i}-\sum _{j=1}^{n}X_{ij}\beta _{j}\right|^{2} $ = $ (\mathbf {y} -\mathbf {X} \mathbf {b} )^{\mathtt {T}}\,\mathbf {W}^{-1}(\mathbf {y} -\mathbf {X} \mathbf {b}) $ <br>

where $W$ is a diagonal matrix (also the cov-var matrix of residuals) to be modelled.

In [1]:
def wls_reg(X,Y):
    X = sm.add_constant(X)
    model = regression.linear_model.OLS(Y, X).fit()
    model_1 = sm.WLS(Y,X, weights=1/(model.resid**2)).fit() #model the weight: higher error > lower weight
    return model_1

**Robust Regression**<br>

${W}$ can be modeled as a function of residuals, which is a function of $\mathbf {X} ,\mathbf {y}, \mathbf {b}$ <br>
${W}$ = $f (\mathbf {X} ,\mathbf {y}, \mathbf {b})$<br>

Also, regression coefficients ${b}$ is a function of $\mathbf {X} ,\mathbf {y}, \mathbf {W}$<br>
${b}$ = $g (\mathbf {X} ,\mathbf {y}, \mathbf {W})$<br>

In Robust Regression, iteration starts with ${W_{0}}$ = $I$ (indeed OLS), then ${W_{0}}$ is put back to $g$ and generate ${b_{o}}$, which is then put to $f$ to generate ${W_{1}}$.<br>
The process is repeated until ${b}$ converges.<br>
Please refer below for different models for the residual weight ${W}$.<br><br>
https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Robust_Regression.pdf

In [ ]:
def robust_reg(X,Y):
    X = sm.add_constant(X)
    model = sm.RLM(Y,X ,M=sm.robust.norms.HuberT()).fit() # select algo
    return model

**Detect Heteroscedasticity (non costant variance of residuals)**<br>

Heteroscedasticity violates one of the assumptions of OLS.

Use breuschpagan test for graually increased residuals.<br>
Use whtie test for graually increased residuals.

In [ ]:
def bp_test(model):
    pval = smd.het_breuschpagan(model.resid, model.model.exog)[1]
    if pval<0.05:
        print("BP test p-val: ",  pval,". Using threshold 5%, the residual is heteroscedastic")
    else:
        print("BP test p-val: ",  pval,". Using threshold 5%, the residual is not heteroscedastic")

In [ ]:
def white_test(model):
    pval = smd.het_white(model.resid, model.model.exog)[1]
    if pval<0.05:
        print("White test p-val: ",  pval,". Using threshold 5%, the residual is heteroscedastic")
    else:
        print("White test p-val: ",  pval,". Using threshold 5%, the residual is not heteroscedastic")

In [ ]:
def ransac(X,Y):
    #x = np.array(data_set)[:,3:4]
    #y = np.array(data_set)[:,0:1]

    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor(residual_threshold=1.5) #larger threshold > less outlier
    ransac.fit(X, Y)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    # Predict data of estimated models
    line_X = np.arange(X.min(), X.max())[:, np.newaxis]
    line_y_ransac = ransac.predict(line_X)

    print("Estimated coefficients (true, linear regression, RANSAC):")
    print(ransac.estimator_.coef_, ransac.estimator_.intercept_)

    plt.plot(line_X, line_y_ransac, color='cornflowerblue', linewidth=2,
             label='RANSAC regressor')
    plt.scatter(X[inlier_mask], Y[inlier_mask], color='yellowgreen', marker='.',
                label='Inliers')
    plt.scatter(X[outlier_mask], Y[outlier_mask], color='gold', marker='.',
                label='Outliers')
    plt.legend(loc='lower right')
    
    return ransac.estimator_.coef_, ransac.estimator_.intercept_, ransac.fit(X, Y).score(X, Y)